In [128]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


Time to scrape from Wikipedia

In [129]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
columns

['Postcode', 'Borough', 'Neighbourhood']

In [130]:
df = pd.DataFrame(columns = columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '')]
    else:
        values = [td.text for td in tds]
    df = df.append(pd.Series(values, index=columns), ignore_index=True)
print(df)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


Will save the dataframe to my local disk just in case, and then read it in:

In [131]:
df.to_csv(r'C:\Users\SM\Desktop\projects\Coursera_Capstone\canada.csv', index=False)

In [132]:
df = pd.read_csv(r'C:\Users\SM\Desktop\projects\Coursera_Capstone\canada.csv', index_col=None)

In [133]:
df.describe()

,Postcode,Borough,Neighbourhood
count,287,287,287
unique,180,11,209
top,M9V,Not assigned,Not assigned
freq,8,77,77


In [134]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Time to remove empty boroughs

In [135]:
df = df[df.Borough != 'Not assigned']

In [136]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


I will now group them and combine neighbourhoods that share the same postcode

In [137]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].unique().apply(', '.join).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [138]:
df.shape

(103, 3)